In [ ]:
# Import functions 
import sys
sys.path.insert(0, '..')
from racecar_utils import *   

In [ ]:
# Start ROS Node
rospy.init_node('racecar')
rc = Racecar()
print('ROS node started successfully')

# Line Following

<p style='font-size:1.75rem;line-height:1.5'>
    Today, we will create an algorithm for the racecar to detect and follow a colored line. We will have several different tape colors for the car to select from. Each path will lead us on a different path adventure!
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'> 
    Depending on the color, the racecar may take a shorter or longer path to its final destination. 
    </p>

## Step 1: Fix the Camera Offset

<p style='font-size:1.75rem;line-height:1.5'>
    The camera is slightly shifted horizontally (x-axis). 
    <br> Update <code>SCREEN_CENTER</code> to re-set the center pixel location of your output image.
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
    The function below draws a circle at <code>SCREEN_CENTER</code>. 
    <br> How to find the correct value:
    <ol style='font-size:1.75rem;line-height:1.5'>
        <li>Place an object directly in front of your camera at where you think center is.</li>
        <li>Run the two cell blocks below to test your <code>SCREEN_CENTER</code> value. </li>
        <li>Your drawn circle should be matched to the center of the object in the outputted image. </li>
    </ol>
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
    Getting the correct <code>SCREEN_CENTER</code> value here will help you later in the the <code>get_angle</code> function.
    </p>

In [ ]:
SCREEN_CENTER = 320     # pixel x-axis, camera is right-shifted

In [ ]:
# mark location of SCREEN_CENTER (x-axis)
def identify_center(img):
    if SCREEN_CENTER < 0 or SCREEN_CENTER > img.shape[1]:
        print('SCREEN_CENTER out of bounds! Your image is: {}'.format(img.shape[:2]))
    cv2.circle(img, (int(SCREEN_CENTER), img.shape[0]/2), 5, (0,255,0), 3)
    return img

# display image
show_image(identify_center)

## Step 2: Crop Image

<p style='font-size:1.75rem;line-height:1.5'>
    It is intuitive for us humans to trace the closest part of the line (the part under our feet) as we walk. However, the car sees a long line ahead of itself and doesn't know which part of the line to follow. We want to limit what the car sees/follows by limiting its field of view.
      </p>

<p style='font-size:1.75rem;line-height:1.5'>
    Modify <code>TOPLEFT_COORD</code> and <code>BOTTOMRIGHT_COORD</code> to crop the image below. The function draws a black rectangle over the top half of the image, showing only the part of the line that is closest to the car.
    </p>

In [ ]:
TOPLEFT_COORD = (0, 400)
BOTTOMRIGHT_COORD = (639, 0)

In [ ]:
#draws rectangle over frame to crop
def crop(img):
    cv2.rectangle(img, TOPLEFT_COORD, BOTTOMRIGHT_COORD, (0, 0, 0), -1) 
    return img

show_image(crop)

## Step 3: Find HSV Range of the Line

<p style='font-size:1.75rem;line-height:1.5'>
    Set the HSV lower and upper bounds to detect the line!
    <ol style='font-size:1.75rem;line-height:1.5'>
        <li>Use <code>hsv_select(&lt;seconds&gt;)</code> to mask for the line!</li>
        <li>Update <code>HSV_LOWER</code> and <code>HSV_UPPER</code>!</li>
    </ol>
    </p>

In [ ]:
VIDEO_TIME = 10
hsv_select_live(VIDEO_TIME) # default: 10 sec

In [ ]:
HSV_LOWER_1 = np.array([None, None, None])
HSV_UPPER_1 = np.array([None, None, None])

## Step 4: <code>get_angle</code>

<p style='font-size:1.75rem;line-height:1.5'>
    Now that we have the point we want to follow, we have to <b>determine what angle to turn the car</b> in order to get to that point.
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    <code>get_angle</code> returns the turn angle of the car. We will determine whether to turn left or right, and how much to turn, depending on the location of the line in our image. When the line is at the center of the image, the car and line are aligned. 
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    This function is similar to the one we wrote for Cone Following, but NOT exactly. 
    <ul style='font-size:1.75rem;line-height:1.5'>
        <li>The center of the line is found at <code>contour_center</code></li>
        <li>Calculate the offset error: 
            <br><code>error = contour_center - float(SCREEN_CENTER)</code></li>
        <li>Calculate the ratio that the line is away from the center of the screen: 
            <br><code>ratio = error / SCREEN_CENTER</code></li>
        <li>Multiply <code>ratio</code> by the <code>max_angle</code> to find the return angle. </li>
    </ul>
    </p>

In [ ]:
TURN_FACTOR = 30          # max turn angle (in degrees)
DRIVE_SPEED = 0.215       # car speed      

In [ ]:
def get_angle(contour_center):
    max_angle = -abs(TURN_FACTOR) 
    
    # TASK #1: Calculate the offset error


    # TASK #2: Calculate the ratio


    # TASK #3: Calculate the turn angle
 

    # TASK #4: Return the turn angle


## Step 5: <code>get_contour_center</code>

<p style='font-size:1.75rem;line-height:1.5'>
    <code>get_contour_center</code> returns the center of the largest contour (if contours exist). 
    <br>Else, the function returns None. 
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    <code>EXIST_SIZE</code> is the minimum  size required for a contour to be considered. Any contours smaller than <code>EXIST_SIZE</code> are disregarded. 
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    Run the cell block below.
    </p>

In [ ]:
EXIST_SIZE = 30

In [ ]:
def get_contour_center(contours):
    greatest_contour = None
    if len(contours) > 0:
        greatest_contour = max(contours, key = cv2.contourArea) # get largest contour
        if cv2.contourArea(greatest_contour) < EXIST_SIZE:
            greatest_contour = None 
    M = None
    if greatest_contour is not None:
        M = cv2.moments(greatest_contour)
    return M

## Step 6: <code>line_following</code>

<p style='font-size:1.75rem;line-height:1.5'>
    Now, let us write the <code>line_following</code> function below: 
    </p>

In [ ]:
DEBUG = True      # show video

In [ ]:
def line_following(frame):
    frame = crop(frame) #draws rectangle over frame to crop
    
    # TASK #1: Convert frame from BGR to HSV

    
    # TASK #2: Mask the image via HSV_LOWER_1 and HSV_UPPER_1

    
    # TASK #3: Find the contours

     
    # TASK #4: Call 'get_contour_center', and save as 'M'. This function accepts on argument: 'contours'.
    #          This function returns the center of the largest contour (if contours exist), else returns None

    
    if M is not None:
        if M['m00'] != 0:
            x, y = int(M['m10'] / M['m00']), int(M['m01'] / M['m00'])
            contour_center = x
            
            # TASK #5: Draw all the contours 

            
            # TASK #6: Draw a circle at the contour center

            
            # show image
            if DEBUG:
                show_frame(frame)
            
            # TASK #7: Call 'get_angle', giving it 'contour_center' as the argument. Save as 'angle'.

            
            # TASK #8: Call 'rc.drive', with parameters: 'DRIVE_SPEED' and 'angle'


## Step 7: Test our Line Follower!

<p style='font-size:1.75rem;line-height:1.5'>
    Run the block below to test our written functions above!
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    <b style='color:red'>Remember to get a sticker from a TA before taking your car off the block!</b>
    </p>

In [ ]:
# For debugging
TEST_TIME = 10    # test time for autonomous driving

In [ ]:
# setup display
display = IPython.display.display('', display_id=current_display_id)
current_display_id += 1

# run the line_following function!
rc.run(line_following, TEST_TIME)

## Step 8: Parameter Tuning

<p style='font-size:1.75rem;line-height:1.5'>
    Tune the following parameters to improve your line follower!
    <ul style='font-size:1.75rem;line-height:1.5'>
        <li><code>SCREEN_CENTER</code>: to adjust for screen center offset</li>
        <li><code>TOPLEFT_COORD</code> and <code>BOTTOMRIGHT_COORD</code>: to adjust the crop size view area</li>
        <li><code>HSV_LOWER</code> and <code>HSV_UPPER</code>: to adjust the threshold color</li>
        <li><code>TURN_FACTOR</code>: for the turn intensity of the car</li>
        <li><code>DRIVE_SPEED</code>: for the speed of the car</li>
    </ul>
    </p>

# Line Following Competition!!!

<p style='font-size:1.75rem;line-height:1.5'>
    We have prepared a race track, where we will compete against other teams to finish the course in the shortest amount of time!
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
    Refer to the instructors for further instructions in class. 
    <br> Good luck!
    </p>